In [1]:
import Ipynb_importer
from a_csp_darknet53  import *
from b_yolov4_neck import *
from c_yolov4_head_keras import *
from d_prediction import *
from e_model import *
from f_utils import *

importing Jupyter notebook from a_csp_darknet53.ipynb
importing Jupyter notebook from z_layers.ipynb
importing Jupyter notebook from b_yolov4_neck.ipynb
importing Jupyter notebook from c_yolov4_head_keras.ipynb
importing Jupyter notebook from d_prediction.ipynb
importing Jupyter notebook from f_utils.ipynb


In [3]:
import os
import time
from tensorflow.keras.layers import Input, Lambda

In [3]:
default_config = {
        "model_type": 'tiny_yolo3_darknet',
        "weights_path": os.path.join('weights', 'yolov3-tiny.h5'),
        "pruning_model": False,
        "anchors_path": os.path.join('configs', 'tiny_yolo3_anchors.txt'),
        "classes_path": os.path.join('configs', 'coco_classes.txt'),
        "score" : 0.1,
        "iou" : 0.4,
        "model_image_size" : (416, 416),
        "elim_grid_sense": False,
        "gpu_num" : 1,
    }


In [4]:
default_config = {
        "weights_path"        : 'model_data/yolo4_weight.h5',
        "anchors_path"     : 'model_data/yolo_anchors.txt',
        "classes_path"      : 'model_data/coco_classes.txt',
        "score"             : 0.5,
        "iou"                  : 0.3,
        "eager"             : False,
        "max_boxes"         : 100,
        # 显存比较小可以使用416x416
        # 显存比较大可以使用608x608
        "model_image_size"  : (416, 416),
        #---------------------------------------------------------------------#
        #   该变量用于控制是否使用letterbox_image对输入图像进行不失真的resize，
        #   在多次测试后，发现关闭letterbox_image直接resize的效果更好
        #---------------------------------------------------------------------#
        "letterbox_image"   : False,
    }

In [4]:
def get_yolo3_inference_model(model_type, anchors, num_classes, weights_path=None,
                              input_shape=None, confidence=0.1, iou_threshold=0.4, elim_grid_sense=False):
    """create the inference model, for YOLOv3"""
    num_anchors = len(anchors)
    num_feature_layers = num_anchors //3
    
    image_shape = Input(shape=(2,), dtype='int64', name='image_shape')
    # TODO: get_yolo3_mode
    model_body, _ = get_yolo3_model(model_type, num_feature_layers, num_anchors, num_classes, input_shape=input_shape)
    
    if weights_path:
        model_body.load_weights(weights_path, by_name=False)
        print('Load weight {}'.format(weights_path))
        
    # TODO: batch_yolo_postprocess
    boxes, scores, classes = Lambda(batch_yolo_postprocess, name='yolo3_postprocess',
                                    arguments={
                                        'anchors':anchors, 
                                        'num_classes':num_classes, 
                                        'confidence':confidence,
                                        'iou_threshold': iou_threshold,
                                        'elim_grid_sense':elim_grid_sense
                                    })([*model_body.output, image_shape])
    model = Model([model_body.input, image_shape], [boxes, scores, classes])
    
    return model
    
                                        

SyntaxError: invalid syntax (<ipython-input-4-a4a94f7fbf0a>, line 18)

In [ ]:
def YOLO(object):
    _defaults = default_config
    
    # classmethod 修饰符对应的函数不需要实例化，不需要 self 参数，
    # 但第一个参数需要是表示自身类的 cls 参数，可以来调用类的属性，类的方法，实例化对象等。
    @classmethod
    def get_defaults(cls, n):
        if n in cls._defaults:
            return cls._defaults[n]
        else:
            return "Unrecognized attribute name' " + n + " ' " 
        
    def __init__(self, **kwargs):
        super(YOLO, self).__init__()
        self.__dict__.update(self._defaults)  # 还能这样？
        self.__dict__.update(kwargs)  # update with user overrides
        self.class_names = get_classes(self.classes_path)
        self.anchors = get_anchors(self.anchors_path)
        self.colors = get_colors(self.class_names)
        self.inference_mode = self._generate_model()
    
    def _generate_model(self):
        ''''''
        weights_path = os.path.expanduser(self.weights_path)
        assert weights_path.endswith('.h5'), 'Keras model or weights must be a .h5 file.'
        
        # 先验框数量和种类数量
        num_anchors = len(self.anchors)
        num_classes = len(self.class_names)
        
        # 载入模型
        # TODO: get_yolo3_inference_model
        inference_model = get_yolo3_inference_model(Input(shape=(None,None,3)), num_anchors//3, num_classes)
        
        inference_model.summary()
        return inference_model
    
    
    def predict(self, image_data, image_shape):
        out_boxes, out_scores, out_classes = self.inference_model.predict([image_data, Image_shape])
        
        out_boxes = out_boxes[0]
        out_scores = out_scores[0]
        out_classes = out_classes[0]
        
        out_boxes = out_boxes.astype(np.int32)
        out_classes = out_classes.astype(np.int32)
        return out_boxes, out_classes, out_scores
        
        
    def detect_image(self, image):
        if self.model_image_size != (None, None):
            assert self.model_image_size[0]%32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1]%32 == 0, 'Multiples of 32 required'
            
        image_data = preprocess_image(image, self.model_image_size)
        
        # 
        image_shape = np.array([image.size[1], image.size[0]])
        image_shape = np.expand_dims(image_shape, 0)
        
        start = time.time()
        out_boxes, out_classes, out_scores = self.predict(image_data, image_shape)
        end = time.time()
        
        print('Found {} boxes for {}'.format(len(out_boxes), 'img'))
        print("Inference time: {:.8f}s".format(end - start))
        
        #draw result on input image
        image_array = np.array(image, dtype='uint8')
        image_array = draw_boxes(image_array, out_boxes, out_classes, out_scores, self.class_names, self.colors)

        out_classnames = [self.class_names[c] for c in out_classes]
        return Image.fromarray(image_array), out_boxes, out_classnames, out_scores
    
    
    
    